# Passo 2: Análise Exploratória Geral (Descriptive Analytics)

## Objetivo
Este notebook tem como foco a análise descritiva dos dados de varejo. Antes de entrarmos em análises complexas de elasticidade e causalidade, precisamos entender:
1.  **Quem são os campeões de vendas?** (Pareto: Receita e Volume)
2.  **Como as vendas se comportam ao longo do tempo?** (Sazonalidade e Tendência Geral)
3.  **Qual a distribuição de preços e categorias?**
4.  **Qualidade dos dados** (visão geral das distribuições).

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings

warnings.filterwarnings("ignore")
sns.set_style("whitegrid")
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Cores Premium
COLORS = ['#2E86C1', '#28B463', '#E74C3C', '#F1C40F', '#8E44AD']

## 1. Carregamento e Visão Geral

In [0]:
# [AUTO-UPDATED] Load from Spark Table
from pyspark.sql import SparkSession
import pandas as pd
import os

# Initialize Spark
spark = SparkSession.builder.appName("EDA").config("spark.sql.warehouse.dir", "spark-warehouse").getOrCreate()
# Carregar dados
df = spark.read.table('elastic_price.raw.retail_price')
# Mostrar dados
display(df)

## 2. Top Produtos (Best Sellers)
Quem gera mais receita? Quem vende mais unidades?

In [0]:
# Agrupamento por Produto
prod_agg = df.groupby('product_id').agg({
    'total_revenue': 'sum',
    'qty': 'sum',
    'unit_price': 'mean', # Preço médio
    'product_category_name': 'first'
}).reset_index()

# Top 10 por Receita
top_rev = prod_agg.sort_values('total_revenue', ascending=False).head(10)

fig = px.bar(top_rev, x='product_id', y='total_revenue', 
             color='product_category_name', 
             title='Top 10 Produtos por Receita Total',
             text_auto='.2s',
             template='plotly_white')
fig.update_layout(xaxis_title="Produto", yaxis_title="Receita ($)")
fig.show()

In [0]:
# Top 10 por Volume (Qtd)
top_qty = prod_agg.sort_values('qty', ascending=False).head(10)

fig = px.bar(top_qty, x='product_id', y='qty', 
             color='product_category_name', 
             title='Top 10 Produtos por Volume de Vendas (Qtd)',
             text_auto=True,
             template='plotly_white')
fig.update_layout(xaxis_title="Produto", yaxis_title="Quantidade Vendida")
fig.show()

## 3. Histórico de Vendas (Evolução Temporal)
Como a receita total da empresa evoluiu mês a mês?

In [0]:
# Agrupamento Mensal Global
df_time = df.set_index('date').resample('MS').agg({
    'total_revenue': 'sum',
    'qty': 'sum'
}).reset_index()

fig = px.line(df_time, x='date', y='total_revenue', 
              title='Evolução da Receita Mensal Global',
              markers=True,
              line_shape='spline',
              template='plotly_white')
fig.update_traces(line_color='#2E86C1', line_width=3)
fig.update_layout(xaxis_title="Data", yaxis_title="Receita Total")
fig.show()

### 3.1 Vazamento de Tendência por Categoria
Quais categorias estão crescendo?

In [0]:
df_cat_time = df.groupby(['date', 'product_category_name'])['total_revenue'].sum().reset_index()

fig = px.area(df_cat_time, x='date', y='total_revenue', color='product_category_name',
              title='Evolução da Receita por Categoria (Área)',
              template='plotly_white')
fig.show()

## 4. Análise de Preços e Categorias
Distribuição dos preços praticados.

In [0]:
plt.figure(figsize=(10, 6))
sns.histplot(df['unit_price'], bins=50, kde=True, color='teal')
plt.title('Distribuição de Preços Unitários')
plt.xlabel('Preço ($)')
plt.ylabel('Frequência')
plt.show()

In [0]:
# Boxplot de Preço por Categoria
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='product_category_name', y='unit_price', palette='viridis')
plt.title('Dispersão de Preços por Categoria')
plt.xticks(rotation=45)
plt.yscale('log') # Escala Log para visualizar melhor a diferença
plt.ylabel('Preço Unitário (Log Scale)')
plt.show()

## 5. Conclusão da Análise Descritiva
Esta visão geral nos permite identificar:
- Os produtos "Geração de Caixa" (Top Sellers).
- A sazonalidade do negócio.
- As categorias com maior ticket médio.

**Próximo Passo:** No notebook `03_eda_elasticity`, aprofundaremos na relação Preço x Demanda (Elasticidade) e Decomposição de Séries Temporais para o produto principal.